# **Notebook RNN**

In [83]:
import numpy as np
import pandas as pd
import re
from collections import Counter
import pickle
import random

from helpers.dataset import Dataset
from helpers.activation import TanhActivation
from helpers.losses import BinaryCrossEntropy
from helpers.metrics import accuracy
from helpers.activation import ReLUActivation
from rnn_model import RNN

**Nova Classe Optimizer**

In [84]:
class Optimizer:
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.velocity = {}  # Dicionário para armazenar velocidades dos gradientes
        self.learning_rate = learning_rate
        self.momentum = momentum

    def update(self, param, grad):
        """Atualiza os pesos usando Gradient Descent com Momentum"""

        param_id = id(param)  # 🔹 Usar ID único do numpy array

        if param_id not in self.velocity:
            self.velocity[param_id] = np.zeros_like(grad)

        # Atualização com momentum
        self.velocity[param_id] = self.momentum * self.velocity[param_id] + (1 - self.momentum) * grad
        return param - self.learning_rate * self.velocity[param_id]  # 🔹 Retorna os novos pesos



### **Tratamento de Dados**

- Transformar cada palavra num token
(de forma a manter a ordem das palavras)
- Fazer o padding
- Dar print ao dataset a cada mudança (5 primeiras linhas)

Possibilidades:
- Fazer normalizaçao
- Ver possiveis melhorias com chat

**Análise Inicial do Dataset1**

In [85]:
# Definir os caminhos dos arquivos de TREINO
# input_csv1 = "../tarefa_1/clean_input_datasets/gpt_vs_human_data_set_inputs.csv"
# output_csv1 = "../tarefa_1/clean_output_datasets/gpt_vs_human_data_set_outputs.csv"

input_csv1 = "../tarefa_1/clean_input_datasets/ai_human_input_sm.csv"
output_csv1 = "../tarefa_1/clean_output_datasets/ai_human_output_sm.csv"

# Definir os caminhos dos arquivos de TESTE
# input_csv2 = "../tarefa_1/clean_input_datasets/ai_human_input_sm.csv"
# output_csv2 = "../tarefa_1/clean_output_datasets/ai_human_output_sm.csv"

input_csv2 = "../tarefa_1/clean_input_datasets/gpt_vs_human_data_set_inputs.csv"
output_csv2 = "../tarefa_1/clean_output_datasets/gpt_vs_human_data_set_outputs.csv"
 
# Carregar os datasets de treino
df_input1 = pd.read_csv(input_csv1, sep="\t")  # Ajuste o separador se necessário
df_output1 = pd.read_csv(output_csv1, sep="\t")

# Carregar os datasets de teste
df_input2 = pd.read_csv(input_csv2, sep="\t")
df_output2 = pd.read_csv(output_csv2, sep="\t")

# Assumindo que há uma coluna de ID para junção
df_train = pd.merge(df_input1, df_output1, on="ID")
df_test = pd.merge(df_input2, df_output2, on="ID")

# Concatenar treino e teste
df_dataset1_merged = pd.concat([df_train, df_test], ignore_index=True)

# # Carregar os datasets
# df_input = pd.read_csv(input_csv, sep="\t")  # Ajuste o separador se necessário
# df_output = pd.read_csv(output_csv, sep="\t")

# # Mostrar as primeiras 5 linhas do input
# print("Input CSV - Primeiras 5 linhas:")
# print(df_input.head())

# # Mostrar as primeiras 5 linhas do output
# print("\nOutput CSV - Primeiras 5 linhas:")
# print(df_output.head())

# # Juntar os datasets pelo ID
# df_dataset1_merged = pd.merge(df_input, df_output, on="ID")

# Mostrar as primeiras 5 linhas do dataset completo
print("\nDataset Completo - Primeiras 5 linhas:")
print(df_dataset1_merged.head())

print("\nDataset Completo - Ultimas 5 linhas:")
print(df_dataset1_merged.tail())


Dataset Completo - Primeiras 5 linhas:
   ID                                               Text Label
0   1  Cars. Cars have been around since they became ...    AI
1   2  Transportation is a large necessity in most co...    AI
2   3  "America's love affair with it's vehicles seem...    AI
3   4  How often do you ride in a car? Do you drive a...    AI
4   5  Cars are a wonderful thing. They are perhaps o...    AI

Dataset Completo - Ultimas 5 linhas:
         ID                                               Text  Label
34048  4048  This research paper investigates the vortex dy...     AI
34049  4049  Given a remarkable representation of the gener...  Human
34050  4050  The Veldkamp space of two-qubits is a mathemat...     AI
34051  4051  The equilibration of macroscopic degrees of fr...  Human
34052  4052  This research paper investigates the fusion pr...     AI


**Remover caracteres especiais e pontuação e Converter em minúsculas**

In [86]:
# Função para limpar texto
def clean_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remover pontuação
    return text

df_dataset1_merged["clean_text"] = df_dataset1_merged["Text"].apply(clean_text)

# Manter apenas as colunas desejadas e renomear clean_text para Text
df_dataset1_merged = df_dataset1_merged[["ID", "clean_text", "Label"]].rename(columns={"clean_text": "Text"})

print("Texto limpo - primeiras 5 linhas:")
print(df_dataset1_merged.head())

Texto limpo - primeiras 5 linhas:
   ID                                               Text Label
0   1  cars cars have been around since they became f...    AI
1   2  transportation is a large necessity in most co...    AI
2   3  americas love affair with its vehicles seems t...    AI
3   4  how often do you ride in a car do you drive a ...    AI
4   5  cars are a wonderful thing they are perhaps on...    AI


**Remover stopwords (opcional)**

In [87]:
# Lista de stopwords comuns (podes adicionar mais conforme necessário)
stopwords = {
    "the", "of", "and", "in", "to", "is", "a", "that", "for", "are", "on", "with", 
    "as", "at", "by", "from", "this", "it", "an", "be", "or", "which", "was", "were"
}

# Função para remover stopwords
def remove_stopwords(text):
    words = text.split()  # Dividir em palavras
    filtered_words = [word for word in words if word not in stopwords]  # Remover stopwords
    return " ".join(filtered_words)  # Juntar as palavras de novo

# Aplicar ao dataset
df_dataset1_merged["Text"] = df_dataset1_merged["Text"].apply(remove_stopwords)

# Exibir as primeiras 5 linhas após remoção de stopwords
print("Texto sem stopwords - primeiras 5 linhas:")
print(df_dataset1_merged.head())



Texto sem stopwords - primeiras 5 linhas:
   ID                                               Text Label
0   1  cars cars have been around since they became f...    AI
1   2  transportation large necessity most countries ...    AI
2   3  americas love affair its vehicles seems coolin...    AI
3   4  how often do you ride car do you drive one any...    AI
4   5  cars wonderful thing they perhaps one worlds g...    AI


**Criar Embeddings e Label Encoding**

In [88]:
# Mapear labels para valores numéricos
label_map = {"Human": 0, "AI": 1}
df_dataset1_merged["Label"] = df_dataset1_merged["Label"].map(label_map)

# Carregar o GloVe
EMBEDDING_DIM = 50  # Dimensão do embedding

embedding_dict = {}
with open("helpers/glove.6B.50d.txt", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        embedding_dict[word] = vector

print(f"Total de palavras carregadas do GloVe: {len(embedding_dict)}")

# Converter palavras para embeddings
def text_to_embedding(text, embedding_dict, embedding_dim=50):
    words = text.split()
    embeddings = [embedding_dict.get(word, np.zeros(embedding_dim)) for word in words]  # Usa vetor do GloVe ou vetor zerado
    
    # Se a lista estiver vazia, retorna um vetor de zeros
    if len(embeddings) == 0:
        embeddings = [np.zeros(embedding_dim)]

    return embeddings

df_dataset1_merged["Embedding"] = df_dataset1_merged["Text"].apply(lambda x: text_to_embedding(x, embedding_dict, EMBEDDING_DIM))

# Padronizar comprimento das sequências (Padding ou Truncamento)
MAX_SEQUENCE_LENGTH = 200  # Ajustável conforme necessário

# def pad_embedding_sequence(seq, max_length, embedding_dim):
#     if len(seq) > max_length:
#         return np.array(seq[:max_length])  # Truncar
#     else:
#         padding = np.zeros((max_length - len(seq), embedding_dim))  # Criar padding
#         return np.vstack([seq, padding])  # Adicionar padding no final

def pad_embedding_sequence(seq, max_length, embedding_dim):
    seq = np.array(seq)  # Converter para NumPy array para evitar problemas de formato
    
    if seq.shape[0] == 0:  # Se for uma sequência vazia, criar um array de zeros
        seq = np.zeros((1, embedding_dim))

    if seq.shape[0] > max_length:  # Truncar se for maior que o máximo permitido
        return seq[:max_length]

    padding = np.zeros((max_length - seq.shape[0], embedding_dim))  # Criar padding
    return np.vstack([seq, padding])  # Adicionar padding no final


df_dataset1_merged["Embedding"] = df_dataset1_merged["Embedding"].apply(lambda x: pad_embedding_sequence(x, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))

# Converter para array NumPy para alimentar o modelo
X = np.array(df_dataset1_merged["Embedding"].tolist())
y = np.array(df_dataset1_merged["Label"])  # Labels numéricos

print("Formato final dos dados para o modelo:", X.shape)  # Deve ser (n_amostras, 100, 50)

# Manter apenas as colunas desejadas e renomear clean_text para Text
df_dataset1_merged = df_dataset1_merged[["ID", "Embedding", "Label"]].rename(columns={"Embedding": "Text"})

print("Dataset após embedding - primeiras 5 linhas:")
print(df_dataset1_merged.head())

Total de palavras carregadas do GloVe: 400000
Formato final dos dados para o modelo: (34053, 200, 50)
Dataset após embedding - primeiras 5 linhas:
   ID                                               Text  Label
0   1  [[0.5172399878501892, -0.26030999422073364, 1....      1
1   2  [[0.38842999935150146, 0.1005999967455864, 0.6...      1
2   3  [[0.3543199896812439, 0.12161999940872192, -0....      1
3   4  [[0.6893799901008606, -0.1064400002360344, 0.1...      1
4   5  [[0.5172399878501892, -0.26030999422073364, 1....      1


**Converter palavras em números (Tokenização)**

In [89]:
# # Criar vocabulário
# word_counts = Counter()
# for text in df_dataset1_merged["Text"]:
#     word_counts.update(text.split())

# # Definir vocabulário com um limite de palavras (ex: 5000 palavras mais frequentes)
# MAX_VOCAB_SIZE = 10000
# vocab = {word: idx + 1 for idx, (word, _) in enumerate(word_counts.most_common(MAX_VOCAB_SIZE))}

# # Adicionar token especial para palavras desconhecidas
# vocab["<OOV>"] = len(vocab) + 1  

# print("Exemplo de vocabulário:", list(vocab.items())[:10])

# # Converter frases para sequências numéricas
# def text_to_sequence(text, vocab):
#     return [vocab.get(word, vocab["<OOV>"]) for word in text.split()]

# # Aplicar a tokenização ao DataFrame
# df_dataset1_merged["tokenized_text"] = df_dataset1_merged["Text"].apply(lambda x: text_to_sequence(x, vocab))

# # Manter apenas as colunas desejadas e renomear clean_text para Text
# df_dataset1_merged = df_dataset1_merged[["ID", "tokenized_text", "Label"]].rename(columns={"tokenized_text": "Text"})

# print("Texto tokenizado - primeiras 5 linhas:")
# print(df_dataset1_merged.head())

**Padronizar o comprimento das sequências**

tentar padding a esquerda

In [90]:
# Definir tamanho máximo das sequências
MAX_SEQUENCE_LENGTH = 120  # Ajusta conforme necessário
EMBEDDING_DIM = 50  # Mantém o mesmo tamanho dos embeddings

# Função para aplicar padding corretamente
def pad_embedding_sequence(seq, max_length, embedding_dim):
    seq = np.array(seq)  # Garante que a sequência é um array NumPy

    if len(seq) > max_length:
        return seq[:max_length]  # Truncar se for maior
    else:
        padding = np.zeros((max_length - len(seq), embedding_dim))  # Criar padding de vetores de zeros
        return np.vstack([seq, padding])  # Adicionar padding no final

# Aplicar padding às sequências de embeddings
df_dataset1_merged["Text"] = df_dataset1_merged["Text"].apply(lambda x: pad_embedding_sequence(x, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))

# Converter para NumPy array para treinar o modelo
X = np.array(df_dataset1_merged["Text"].tolist())
y = np.array(df_dataset1_merged["Label"])  # Labels numéricos

print("Formato final dos dados para o modelo:", X.shape)  # Deve ser (n_amostras, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)

# Exibir primeiras 5 linhas após padding
print("Texto padronizado - primeiras 5 linhas:")
print(df_dataset1_merged.head())


Formato final dos dados para o modelo: (34053, 120, 50)
Texto padronizado - primeiras 5 linhas:
   ID                                               Text  Label
0   1  [[0.5172399878501892, -0.26030999422073364, 1....      1
1   2  [[0.38842999935150146, 0.1005999967455864, 0.6...      1
2   3  [[0.3543199896812439, 0.12161999940872192, -0....      1
3   4  [[0.6893799901008606, -0.1064400002360344, 0.1...      1
4   5  [[0.5172399878501892, -0.26030999422073364, 1....      1


**Criar Embeddings e Label Encoding**

In [91]:
# # Mapear labels para valores numéricos
# label_map = {"Human": 0, "AI": 1}
# df_dataset1_merged["Label"] = df_dataset1_merged["Label"].map(label_map)

# # Converter para array NumPy
# X = np.array(df_dataset1_merged["Text"].tolist())  # Lista de tokens (já com padding)
# y = np.array(df_dataset1_merged["Label"])  # Labels numéricos

# print("Formato dos dados de entrada:", X.shape)  # Deve ser (n_amostras, 100)

# EMBEDDING_DIM = 50  # Número de dimensões dos embeddings
# VOCAB_SIZE = len(vocab) + 1  # O tamanho do vocabulário + 1 para <OOV>

# # Carregar o GloVe
# embedding_dict = {}
# with open("helpers/glove.6B.50d.txt", "r", encoding="utf-8") as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         vector = np.array(values[1:], dtype="float32")
#         embedding_dict[word] = vector

# # Criar a matriz de embeddings (substituir random embeddings)
# embedding_matrix = np.random.uniform(-1, 1, (VOCAB_SIZE, EMBEDDING_DIM))  # Inicializa aleatoriamente
# for word, idx in vocab.items():
#     if word in embedding_dict:  # Se a palavra estiver no GloVe, usa o vetor pré-treinado
#         embedding_matrix[idx] = embedding_dict[word]

# # Converter tokens para embeddings
# X_embedded = np.array([[embedding_matrix[token] for token in seq] for seq in X])

# print("Formato dos dados com embeddings:", X_embedded.shape)  # Deve ser (n_amostras, 100, EMBEDDING_DIM)

# # Adicionar os embeddings ao DataFrame para visualizar melhor
# df_dataset1_merged["Embedding"] = list(X_embedded)

# # Manter apenas as colunas desejadas e renomear Embedding para Text
# df_dataset1_merged = df_dataset1_merged[["ID", "Embedding", "Label"]].rename(columns={"Embedding": "Text"})

# # Print do dataframe atualizado
# print("\nDataset atualizado - Primeiras 5 linhas:")
# print(df_dataset1_merged.head())



**Normalização dos Embeddings**

dar print as ultimas possiçoes, deve ter 0s


In [92]:
# Função para normalizar cada embedding (zero mean, unit variance)
def normalize_embedding(emb):
    mean = np.mean(emb, axis=0)  # Média por dimensão do embedding
    std = np.std(emb, axis=0) + 1e-8  # Desvio padrão (evita divisão por zero)
    return (emb - mean) / std

# Aplicar normalização alternativa aos embeddings
df_dataset1_merged["Text"] = df_dataset1_merged["Text"].apply(normalize_embedding)

# Converter para array NumPy para treinar o modelo
X = np.array(df_dataset1_merged["Text"].tolist())
y = np.array(df_dataset1_merged["Label"])  # Labels numéricos

print("Formato final dos dados para o modelo:", X.shape)  # Deve ser (n_amostras, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)

# Print do dataset atualizado
print("\nDataset após normalização dos embeddings:")
print(df_dataset1_merged.head())


Formato final dos dados para o modelo: (34053, 120, 50)

Dataset após normalização dos embeddings:
   ID                                               Text  Label
0   1  [[0.5612960064685145, -0.8805426099595246, 2.0...      1
1   2  [[-0.21994956121576564, 0.13893814962304316, 0...      1
2   3  [[0.07885863917045757, 0.030814049393885327, -...      1
3   4  [[0.7642999363812941, -0.4599518049576957, -0....      1
4   5  [[0.5979321950261304, -0.5366264554735934, 2.8...      1


**Drop da coluna ID**

In [93]:
if "ID" in df_dataset1_merged.columns:
    df_dataset1_merged = df_dataset1_merged.drop(columns=["ID"])

print("Formato final dos dados para o modelo:", X.shape)  # Deve ser (n_amostras, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)

# Print do dataset atualizado
print("\nDataset após drop:")
print(df_dataset1_merged.head())

Formato final dos dados para o modelo: (34053, 120, 50)

Dataset após drop:
                                                Text  Label
0  [[0.5612960064685145, -0.8805426099595246, 2.0...      1
1  [[-0.21994956121576564, 0.13893814962304316, 0...      1
2  [[0.07885863917045757, 0.030814049393885327, -...      1
3  [[0.7642999363812941, -0.4599518049576957, -0....      1
4  [[0.5979321950261304, -0.5366264554735934, 2.8...      1


**Divisão do Dataset (Train/Test Split) 70%**

In [ ]:
# # Definir seed global para garantir reprodutibilidade
# SEED = 42
# np.random.seed(SEED)
# random.seed(SEED)

# # Embaralhar o dataset antes de dividir
# df_dataset1_merged = df_dataset1_merged.sample(frac=1, random_state=SEED).reset_index(drop=True)

# # Definir proporções de treino, validação e teste
# train_ratio = 0.7  
# val_ratio = 0.15  # 15% para validação
# test_ratio = 0.15  # 15% para teste

# # Definir índices para divisão
# train_index = int(len(df_dataset1_merged) * train_ratio)
# val_index = train_index + int(len(df_dataset1_merged) * val_ratio)

# # Separar os datasets
# df_train = df_dataset1_merged.iloc[:train_index]
# df_val = df_dataset1_merged.iloc[train_index:val_index]
# df_test = df_dataset1_merged.iloc[val_index:]

# # Print dos datasets separados
# print(f"Tamanho do conjunto de treino: {df_train.shape}")
# print(f"Tamanho do conjunto de validação: {df_val.shape}")
# print(f"Tamanho do conjunto de teste: {df_test.shape}")

# # Converter para arrays NumPy
# X_train, y_train = np.array(df_train["Text"].tolist()), np.array(df_train["Label"])
# X_val, y_val = np.array(df_val["Text"].tolist()), np.array(df_val["Label"])
# X_test, y_test = np.array(df_test["Text"].tolist()), np.array(df_test["Label"])

# print(f"Formato dos dados - Treino: {X_train.shape}, Validação: {X_val.shape}, Teste: {X_test.shape}")



# Definir seed global para garantir reprodutibilidade
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

######################################################### dataset de teste
# Separar as últimas 5 linhas para avaliação final
df_eval_final = df_dataset1_merged.tail(4053)

# Remover essas 5 linhas do dataset antes de embaralhar
df_remaining = df_dataset1_merged.iloc[:-4053]
#########################################################

# Embaralhar o dataset restante
df_remaining = df_remaining.sample(frac=1, random_state=SEED).reset_index(drop=True)

# Definir proporções de treino (70%), validação (15%) e teste (15%)
train_ratio = 0.7
val_ratio = 0.15  # 15% validação
test_ratio = 0.15  # 15% teste

# Definir índices para divisão
train_index = int(len(df_remaining) * train_ratio)
val_index = train_index + int(len(df_remaining) * val_ratio)

# Separar os conjuntos de treino, validação e teste
df_train = df_remaining.iloc[:train_index]
df_val = df_remaining.iloc[train_index:val_index]
df_test = df_remaining.iloc[val_index:]

# Print dos tamanhos dos datasets
print(f"Tamanho do conjunto de treino: {df_train.shape}")
print(f"Tamanho do conjunto de validação: {df_val.shape}")
print(f"Tamanho do conjunto de teste: {df_test.shape}")
print(f"Tamanho do conjunto de avaliação final: {df_eval_final.shape}")

# Converter para arrays NumPy
X_train, y_train = np.array(df_train["Text"].tolist()), np.array(df_train["Label"])
X_val, y_val = np.array(df_val["Text"].tolist()), np.array(df_val["Label"])
X_test, y_test = np.array(df_test["Text"].tolist()), np.array(df_test["Label"])
X_eval_final, y_eval_final = np.array(df_eval_final["Text"].tolist()), np.array(df_eval_final["Label"])

# Print dos formatos dos dados
print(f"Formato dos dados:")
print(f"   Treino: {X_train.shape}")
print(f"   Validação: {X_val.shape}")
print(f"   Teste: {X_test.shape}")
print(f"   Avaliação final: {X_eval_final.shape}")



Tamanho do conjunto de treino: (21000, 2)
Tamanho do conjunto de validação: (4500, 2)
Tamanho do conjunto de teste: (4500, 2)
Tamanho do conjunto de avaliação final: (4053, 2)
Formato dos dados:
  🔹 Treino: (21000, 120, 50)
  🔹 Validação: (4500, 120, 50)
  🔹 Teste: (4500, 120, 50)
  🔹 Avaliação final: (4053, 120, 50)


**Verificação Final do Dataset**

In [ ]:
print("\n Primeiras 5 entradas do conjunto de TREINO:")
print(df_train.head())

print("\n Primeiras 5 entradas do conjunto de VALIDAÇÃO:")
print(df_val.head())

print("\n Primeiras 5 entradas do conjunto de TESTE:")
print(df_test.head())

print("\n Primeiras 5 entradas do conjunto de AVALIAÇÃO FINAL:")
print(df_eval_final.head())

##################################################################

print("\n Primeiras 5 entradas de X_train:", X_train[:5])
print("\n Primeiras 5 entradas de y_train:", y_train[:5])

print("\n Primeiras 5 entradas de X_val:", X_val[:5])
print("\n Primeiras 5 entradas de y_val:", y_val[:5])

print("\n Primeiras 5 entradas de X_test:", X_test[:5])
print("\n Primeiras 5 entradas de y_test:", y_test[:5])

print("\n Primeiras 5 entradas de X_eval_final:", X_eval_final[:5])
print("\n Primeiras 5 entradas de y_eval_final:", y_eval_final[:5])



📌 Primeiras 5 entradas do conjunto de TREINO:
                                                Text  Label
0  [[-0.661526360482493, 1.7914446075613137, 0.53...      0
1  [[0.6163854, 0.819715, -0.69918394, -0.5064302...      0
2  [[-2.130435077862449, 1.2084377298346007, -1.1...      0
3  [[0.3332962829897395, 0.9475958753066612, -0.1...      0
4  [[-1.9136734571069933, -0.554044880262533, 0.4...      1

📌 Primeiras 5 entradas do conjunto de VALIDAÇÃO:
                                                    Text  Label
21000  [[-0.004062520204015047, 0.5759439172082973, -...      1
21001  [[-0.5077380159703886, 1.8100763252645844, 0.0...      0
21002  [[-0.8361131, 0.036529806, -0.0940976, -0.3258...      0
21003  [[-0.572186263545021, 2.059250454773294, 0.377...      1
21004  [[0.38177555798135215, 2.0771762419473268, 0.4...      1

📌 Primeiras 5 entradas do conjunto de TESTE:
                                                    Text  Label
25500  [[0.31073517628748204, 1.491471944512809, 

### **Construção do modelo RNN com código raiz (Sem TensorFlow/SKLearn)**

**Passo 1 - Definição da Estrutura da RNN**

A nossa RNN terá:
- Uma camada de entrada com 100 timesteps e 50 features (nosso embedding).
- Uma camada recorrente com neurônios que aprendem dependências temporais.
- Uma camada de saída para classificação (sigmoid para saída binária).

**Passo 2 - Inicialização de Pesos**

Antes de tudo, vamos definir os pesos da rede:

- W_xh: Pesa a entrada para os neurônios recorrentes.
- W_hh: Pesa as conexões recorrentes.
- W_hy: Pesa a saída do neurônio recorrente para a predição final.
- b_h e b_y: Bias da camada oculta e da saída.

In [96]:
# Definir hiperparâmetros
input_size = 50    # Dimensão dos embeddings
hidden_size = 64   # Número de neurônios na camada oculta
output_size = 1    # Saída binária (0 ou 1)
learning_rate = 0.01  

# Inicializar pesos
np.random.seed(42)  # Para reprodutibilidade
W_xh = np.random.randn(input_size, hidden_size) * 0.01  # Pesos da entrada para a camada oculta
W_hh = np.random.randn(hidden_size, hidden_size) * 0.01 # Pesos da camada oculta para ela mesma
W_hy = np.random.randn(hidden_size, output_size) * 0.01 # Pesos da camada oculta para saída

# Bias
b_h = np.zeros((1, hidden_size))
b_y = np.zeros((1, output_size))

print("Pesos e Biases inicializados!")

Pesos e Biases inicializados!


**Passo 3 - Forward Propagation**

In [97]:
# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))

# def forward_rnn(X):
#     """
#     Executa a propagação para frente da RNN
#     X: (batch_size, sequence_length, input_size) -> (21, 100, 50)
#     """
#     batch_size, seq_length, _ = X.shape
#     h_t = np.zeros((batch_size, hidden_size))  # Inicializar estados ocultos com 0

#     # Percorrer a sequência temporal
#     for t in range(seq_length):
#         x_t = X[:, t, :]  # Pegar o vetor de embeddings no timestep t
#         h_t = np.tanh(np.dot(x_t, W_xh) + np.dot(h_t, W_hh) + b_h)  # Atualizar estado oculto

#     # Cálculo da saída final
#     y_pred = sigmoid(np.dot(h_t, W_hy) + b_y)
#     return y_pred


**Passo 4 - Função de Custo (Binary Cross-Entropy)**

In [98]:
def binary_cross_entropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-8, 1 - 1e-8)  # 🔹 Evita log(0) ou log(1)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) / y_pred.shape[0]

**Passo 5 - Backpropagation Through Time (BPTT)**

In [99]:
# def backward_rnn(X, y_true, y_pred, h_t):
#     """
#     Executa o cálculo dos gradientes (Backpropagation Through Time)
#     """
#     global W_xh, W_hh, W_hy, b_h, b_y  # 🛠️ Agora garantimos que estamos modificando as variáveis globais

#     batch_size, seq_length, _ = X.shape

#     # Gradiente da saída
#     d_y = (y_pred - y_true) / batch_size  

#     # Gradientes dos pesos
#     d_W_hy = np.dot(h_t.T, d_y)
#     d_b_y = np.sum(d_y, axis=0, keepdims=True)

#     # Inicializar gradientes da camada oculta
#     d_h_t = np.dot(d_y, W_hy.T) * (1 - h_t**2)  # Derivada da tanh

#     d_W_xh = np.zeros_like(W_xh)
#     d_W_hh = np.zeros_like(W_hh)
#     d_b_h = np.zeros_like(b_h)

#     # Backpropagation pela sequência temporal
#     for t in reversed(range(seq_length)):
#         x_t = X[:, t, :]
#         d_W_xh += np.dot(x_t.T, d_h_t)
#         d_W_hh += np.dot(h_t.T, d_h_t)
#         d_b_h += np.sum(d_h_t, axis=0, keepdims=True)

#         d_h_t = np.dot(d_h_t, W_hh.T) * (1 - h_t**2)

#     # Atualizar pesos
#     W_xh -= learning_rate * d_W_xh
#     W_hh -= learning_rate * d_W_hh
#     W_hy -= learning_rate * d_W_hy
#     b_h -= learning_rate * d_b_h
#     b_y -= learning_rate * d_b_y


**Mini-Batches**

In [100]:
num_epochs = 100

def get_mini_batches(X, y, batch_size=16, shuffle=True):
    """Divide os dados em mini-batches."""
    n_samples = X.shape[0]
    indices = np.arange(n_samples)
    if shuffle:
        np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        yield X[indices[start:end]], y[indices[start:end]]


**Otimização de Hiperparâmetros**

In [ ]:
# Função de ativação Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivada da função Sigmoid (útil para backpropagation)
def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)


# Definir pesos corretamente (Xavier Initialization)
W_xh = np.random.randn(input_size, hidden_size) * np.sqrt(1. / input_size)
W_hh = np.random.randn(hidden_size, hidden_size) * np.sqrt(1. / hidden_size)
W_hy = np.random.randn(hidden_size, output_size) * np.sqrt(1. / hidden_size)

HYPERPARAMS = [
    {"epochs": 5, "batch_size": 8, "learning_rate": 0.01, "momentum": 0.9, "bptt_trunc": 2},
    {"epochs": 10, "batch_size": 16, "learning_rate": 0.005, "momentum": 0.95, "bptt_trunc": 3},
    {"epochs": 7, "batch_size": 8, "learning_rate": 0.007, "momentum": 0.8, "bptt_trunc": 2},
]

best_accuracy = 0
best_params = None
best_model = None

# Testando hiperparâmetros
for params in HYPERPARAMS:
    print(f"\nTestando hiperparâmetros: {params}")

    rnn = RNN(
        n_units=20,
        # activation=ReLUActivation(),
        activation=TanhActivation(),
        bptt_trunc=params["bptt_trunc"],
        input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM),
        epochs=params["epochs"],
        batch_size=params["batch_size"],
        learning_rate=params["learning_rate"],
        momentum=params["momentum"],
        loss=BinaryCrossEntropy,
        metric=accuracy
    )

    optimizer = Optimizer(learning_rate=params["learning_rate"])
    rnn.initialize(optimizer)

    for epoch in range(params["epochs"]):
        total_loss = 0
        for X_batch, y_batch in get_mini_batches(X_train, y_train, params["batch_size"]):
            y_pred = rnn.forward_propagation(X_batch)
            #y_pred_final = y_pred[:, -1, :]  # 🔹 Selecionar apenas a última saída
            y_pred_final = sigmoid(y_pred[:, -1, :])  # Aplica Sigmoid na última saída

            loss = binary_cross_entropy(y_batch.reshape(-1, 1), y_pred_final)
            grad_loss = (y_pred_final - y_batch.reshape(-1, 1)) / y_batch.shape[0]

            grad_loss_expanded = np.zeros_like(y_pred)
            grad_loss_expanded[:, -1, :] = grad_loss

            rnn.backward_propagation(grad_loss_expanded)

            total_loss += loss

        print(f"Época {epoch+1}/{params['epochs']} - Loss: {total_loss:.4f}")

    # Avaliação
    preds = rnn.predict(X_val)
    
    # Debug do formato de `preds`
    print(f"Formato de preds: {preds.shape}")

    # Corrigir caso `preds` seja 1D
    if preds.ndim == 1:
        preds = preds[:, np.newaxis]

    acc = accuracy(y_val, preds)

    print(f"Accuracy com esses hiperparâmetros: {acc:.4f}")

    if acc > best_accuracy:
        best_accuracy = acc
        best_params = params
        best_model = rnn

print(f"\nMelhor combinação encontrada: {best_params} com accuracy {best_accuracy:.4f}")




🔍 Testando hiperparâmetros: {'epochs': 5, 'batch_size': 8, 'learning_rate': 0.01, 'momentum': 0.9, 'bptt_trunc': 2}
🏋️ Época 1/5 - Loss: 224.4958
🏋️ Época 2/5 - Loss: 199.5008
🏋️ Época 3/5 - Loss: 167.2965
🏋️ Época 4/5 - Loss: 148.1730
🏋️ Época 5/5 - Loss: 136.8105
Formato de preds: (4500,)
📊 Accuracy com esses hiperparâmetros: 0.8436

🔍 Testando hiperparâmetros: {'epochs': 10, 'batch_size': 16, 'learning_rate': 0.005, 'momentum': 0.95, 'bptt_trunc': 3}
🏋️ Época 1/10 - Loss: 57.3542
🏋️ Época 2/10 - Loss: 56.8938
🏋️ Época 3/10 - Loss: 56.1153
🏋️ Época 4/10 - Loss: 54.2285
🏋️ Época 5/10 - Loss: 51.2881
🏋️ Época 6/10 - Loss: 47.4896
🏋️ Época 7/10 - Loss: 43.2062
🏋️ Época 8/10 - Loss: 39.8838
🏋️ Época 9/10 - Loss: 37.6044
🏋️ Época 10/10 - Loss: 35.8057
Formato de preds: (4500,)
📊 Accuracy com esses hiperparâmetros: 0.8167

🔍 Testando hiperparâmetros: {'epochs': 7, 'batch_size': 8, 'learning_rate': 0.007, 'momentum': 0.8, 'bptt_trunc': 2}
🏋️ Época 1/7 - Loss: 227.8933
🏋️ Época 2/7 - Loss: 

**Passo 6 - Treinar o Modelo Final**

In [ ]:
final_rnn = RNN(
    n_units=20,
    # activation=ReLUActivation(),
    activation=TanhActivation(),
    bptt_trunc=best_params["bptt_trunc"],
    input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM),
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    learning_rate=best_params["learning_rate"],
    momentum=best_params["momentum"],
    loss=BinaryCrossEntropy,
    metric=accuracy
)

final_optimizer = Optimizer(learning_rate=best_params["learning_rate"])
final_rnn.initialize(final_optimizer)

for epoch in range(best_params["epochs"]):
    total_loss = 0
    for X_batch, y_batch in get_mini_batches(X_train, y_train, best_params["batch_size"]):
        y_pred = final_rnn.forward_propagation(X_batch)
        #y_pred_final = y_pred[:, -1, :]  # 🔹 Pegamos apenas a última saída da sequência
        y_pred_final = sigmoid(y_pred[:, -1, :])  # Aplica Sigmoid na última saída

        loss = binary_cross_entropy(y_batch.reshape(-1, 1), y_pred_final)

        # Calcular o gradiente correto
        grad_loss = (y_pred_final - y_batch.reshape(-1, 1)) / y_batch.shape[0]

        # Expandir para 3 dimensões para ser compatível com a RNN
        grad_loss_expanded = np.zeros_like(y_pred)  # (batch_size, timesteps, output_size)
        grad_loss_expanded[:, -1, :] = grad_loss  # Apenas o último timestep recebe gradiente

        # Passar o gradiente expandido
        final_rnn.backward_propagation(grad_loss_expanded)

        total_loss += loss

    print(f"Treino final - Época {epoch+1}/{best_params['epochs']} - Loss: {total_loss:.4f}")

# Testar Modelo Final
y_test_pred = final_rnn.predict(X_test)

print(f"Formato de y_test_pred: {y_test_pred.shape}")  # 🛠️ Debug

# Se for 1D, expandimos para 2D
if y_test_pred.ndim == 1:
    y_test_pred = y_test_pred[:, np.newaxis]

# Se for 2D (batch_size, timesteps), pegamos o último timestep
if y_test_pred.ndim == 2:
    y_test_pred_final = y_test_pred[:, -1]  #  Sem `:` no final, pois já é 1D
else:
    y_test_pred_final = y_test_pred[:, -1, :]  #  Apenas se for 3D

y_test_pred_labels = (y_test_pred_final > 0.5).astype(int)

y_test_true = y_test.flatten()
accuracy = np.mean(y_test_pred_labels == y_test_true)
print(f"\nAccuracy final no conjunto de teste: {accuracy:.4f}")

# Criar DataFrame com Expected vs Predicted
df_results = pd.DataFrame({
    "expected_value": y_test_true,
    "predicted_value_raw": y_test_pred_final.flatten(),  # Valor original antes do arredondamento
    "predicted_value": y_test_pred_labels.flatten()  # Valor final binário (0 ou 1)
})

# Mostrar as previsões para comparação
print("\nComparação entre valores esperados e previstos:")
print(df_results)



🏋️ Treino final - Época 1/5 - Loss: 228.0248
🏋️ Treino final - Época 2/5 - Loss: 216.9531
🏋️ Treino final - Época 3/5 - Loss: 187.3148
🏋️ Treino final - Época 4/5 - Loss: 167.3577
🏋️ Treino final - Época 5/5 - Loss: 151.5369
Formato de y_test_pred: (4500,)

🎯 Accuracy final no conjunto de teste: 0.8378

📊 Comparação entre valores esperados e previstos:
      expected_value  predicted_value_raw  predicted_value
0                  0                  0.0                0
1                  0                  1.0                1
2                  0                  0.0                0
3                  0                  0.0                0
4                  0                  0.0                0
...              ...                  ...              ...
4495               0                  0.0                0
4496               1                  1.0                1
4497               1                  1.0                1
4498               1                  1.0              

**Passo 7 - Avaliação do Modelo com dados do Dataset1**

In [ ]:
# X_eval_final, y_eval_final


# Testar Modelo Final
y_test_pred2 = final_rnn.predict(X_eval_final)

print(f"Formato de y_test_pred2: {y_test_pred2.shape}")  # 🛠️ Debug

# Se for 1D, expandimos para 2D
if y_test_pred2.ndim == 1:
    y_test_pred2 = y_test_pred2[:, np.newaxis]

# Se for 2D (batch_size, timesteps), pegamos o último timestep
if y_test_pred2.ndim == 2:
    y_test_pred_final2 = y_test_pred2[:, -1]  #  Sem `:` no final, pois já é 1D
else:
    y_test_pred_final2 = y_test_pred2[:, -1, :]  #  Apenas se for 3D

y_test_pred_labels2 = (y_test_pred_final2 > 0.5).astype(int)

y_test_true2 = y_eval_final.flatten()
accuracy = np.mean(y_test_pred_labels2 == y_test_true2)
print(f"\nAccuracy final no conjunto de teste: {accuracy:.4f}")

# Criar DataFrame com Expected vs Predicted
df_results = pd.DataFrame({
    "expected_value": y_test_true2,
    "predicted_value_raw": y_test_pred_final2.flatten(),  # Valor original antes do arredondamento
    "predicted_value": y_test_pred_labels2.flatten()  # Valor final binário (0 ou 1)
})

# Mostrar as previsões para comparação
print("\nComparação entre valores esperados e previstos:")
print(df_results)


Formato de y_test_pred2: (4053,)

🎯 Accuracy final no conjunto de teste: 0.4917

📊 Comparação entre valores esperados e previstos:
      expected_value  predicted_value_raw  predicted_value
0                  0                  0.0                0
1                  1                  0.0                0
2                  0                  0.0                0
3                  1                  1.0                1
4                  0                  0.0                0
...              ...                  ...              ...
4048               1                  0.0                0
4049               0                  0.0                0
4050               1                  0.0                0
4051               0                  0.0                0
4052               1                  0.0                0

[4053 rows x 3 columns]


### **Análise de resultados**

**Treino com dataset: gpt_vs_human**

- Durante o treino: 0.87 - 0.9

- Para dataset1: 0.66

- Para dataset2: 0.8 - 1.0

- Para ai_human: 0.51

**Treino com dataset: ai_human**

- Durante o treino: 0.81 - 0.84

- Para gpt_vs_human: 0.49

### **To Do**

sugestão do professor:
- embedding com one hot encoding